# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    betas = P["Beta"]
    amount_invested = P["Amount Invested"]

    weighted_beta = (betas * amount_invested).sum() / amount_invested.sum()

    er_portfolio = rf + weighted_beta * mkt_rp

    return er_portfolio

print(expected_return(portfolio, 0.02, 0.05))

0.08600000000000001


Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    # Write your code here
    amounts = portfolio["Amount Invested"]
    vols = portfolio["Volatility"]

    # Calculate the weights of each asset in the portfolio
    total_investment = amounts.sum()
    if total_investment == 0:
        # Handle the case where total investment is zero to avoid division by zero
        return float('nan')
    
    weights = amounts / total_investment

    # Calculate the portfolio volatility
    port_volatility = (weights[0]**2 * vols[0]**2 + weights[1]**2 * vols[1]**2 +
                       2 * weights[0] * weights[1] * vols[0] * vols[1] * cor)**0.5

    return port_volatility

print(volatility(portfolio, -0.2))

0.24083189157584595


Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    er_market = rf + mkt_rp

    # Calculate the expected return of the portfolio using the CML equation
    er_portfolio = rf + ((er_market - rf) / mkt_vol) * vol

    return er_portfolio
print(expected_return2(0.02, 0.05, 0.16, 0.2408) )

0.09525


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    returns_data = pd.read_csv(return_path)

    # Extract the relevant columns
    returns_data = returns_data[['CUSIP', 'RET', 'year', 'month']]

    # Read the Fama-French factors data from "F-F_Research_Data_Factors.CSV"
    factors_data = pd.read_csv(factors_path)

    # Ensure that the merging columns have the same data type
    returns_data['CUSIP'] = returns_data['CUSIP'].astype(str)
    returns_data['year'] = returns_data['year'].astype(str)
    returns_data['month'] = returns_data['month'].astype(str)

    # Replace 'date' with the correct column name for date in your factors data
    factors_data['YearMonth'] = factors_data['date'].astype(str)

    # Extract year and month from 'YearMonth'
    factors_data['year'] = factors_data['YearMonth'].str[:4]
    factors_data['month'] = factors_data['YearMonth'].str[4:]

    # Ensure that the merging columns have the same data type
    factors_data['year'] = factors_data['year'].astype(str)
    factors_data['month'] = factors_data['month'].astype(str)

    # Merge the two datasets on 'CUSIP', 'year', and 'month'
    merged_data = pd.merge(returns_data, factors_data, on=['year', 'month'], how='inner')

    # Calculate excess returns
    merged_data['Excess_Return'] = merged_data['RET'] - merged_data['RF']

    # Calculate Cov(R_i - Rf, R_mkt - Rf) and Var(R_mkt - Rf)
    cov_matrix = merged_data[['Excess_Return', 'Mkt-RF']].cov()
    cov_rm_rf = cov_matrix.iloc[0, 1]
    var_rm_rf = cov_matrix.iloc[1, 1]

    # Calculate CAPM beta
    beta = cov_rm_rf / var_rm_rf

    # Calculate expected return using CAPM
    expected_return = merged_data['RF'].mean() + beta * merged_data['Mkt-RF'].mean()

    # Create the output DataFrame
    output = pd.DataFrame(data={
        "CUSIP": merged_data['CUSIP'].unique(),  # Use unique CUSIP values
        "EXP_RETURN": expected_return,
        "BETA": beta
    })

    return output


return_path = 'monthlycrsp_mod7.csv'
factors_path = 'F-F_Research_Data_Factors.CSV'

result = CAPM_beta(return_path, factors_path)
print(result)

         CUSIP  EXP_RETURN      BETA
0        36020     0.35898  0.016899
1        36110     0.35898  0.016899
2        88630     0.35898  0.016899
3        91810     0.35898  0.016899
4        91910     0.35898  0.016899
...        ...         ...       ...
3220  46254010     0.35898  0.016899
3221  47650210     0.35898  0.016899
3222   7784210     0.35898  0.016899
3223  19124010     0.35898  0.016899
3224  31806710     0.35898  0.016899

[3225 rows x 3 columns]


Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [7]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    beta_values = data['BETA']
    expected_returns = data['EXP_RETURN']

# Use np.polyfit to fit a linear regression model (degree 1)
    slope = np.polyfit(beta_values, expected_returns, 1)

    return slope
# Example usage with the output from CAPM_beta
slope_value = slope(result)
print(slope_value)

[10.62148832  0.17949003]


C:\Users\saads\AppData\Local\Temp\ipykernel_33156\3079937348.py:11: RankWarning: Polyfit may be poorly conditioned
  slope_value = slope(result)
